In [1]:
import pyspark
print(pyspark.__version__)


3.5.3


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, LongType

spark = SparkSession.builder.appName("MovieDataAnalysis").getOrCreate()


movies_schema = StructType([
    StructField("DirectorName", StringType(), True),
    StructField("Genres", StringType(), True),
    StructField("MovieName", StringType(), True),
    StructField("Budget", LongType(), True),
    StructField("Year", FloatType(), True),
    StructField("Rating", FloatType(), True)
])

user_ratings_schema = StructType([
    StructField("userId", IntegerType(), True),
    StructField("movieId", IntegerType(), True),
    StructField("rating", FloatType(), True),
    StructField("timestamp", LongType(), True)
])


movies_df = spark.read.csv('../../data/movies.csv', header=True, schema=movies_schema)
user_ratings_df = spark.read.csv('../../data/ratings.csv', header=True, schema=user_ratings_schema)

movies_df.show(5)
user_ratings_df.show(5)


+-----------------+--------------------+--------------------+---------+------+------+
|     DirectorName|              Genres|           MovieName|   Budget|  Year|Rating|
+-----------------+--------------------+--------------------+---------+------+------+
|    James Cameron|Action|Adventure|...|              Avatar|237000000|2009.0|   7.9|
|   Gore Verbinski|Action|Adventure|...|Pirates of the Ca...|300000000|2007.0|   7.1|
|       Sam Mendes|Action|Adventure|...|             Spectre|245000000|2015.0|   6.8|
|Christopher Nolan|     Action|Thriller|The Dark Knight R...|250000000|2012.0|   8.5|
|      Doug Walker|         Documentary|Star Wars: Episod...|     NULL|  NULL|   7.1|
+-----------------+--------------------+--------------------+---------+------+------+
only showing top 5 rows

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|1260759144|
|     1|   1029|   3.0|1260759179|
|     1|   1061|   3.0|126

# Truy vấn 1: Số lượng phim của một đạo diễn

In [4]:
director_movie_count = movies_df.groupBy("DirectorName").count()

# Top 10
director_movie_count.orderBy("count", ascending=False).show(10)

+-----------------+-----+
|     DirectorName|count|
+-----------------+-----+
| Steven Spielberg|   26|
|      Woody Allen|   22|
|   Clint Eastwood|   20|
|  Martin Scorsese|   18|
|     Ridley Scott|   17|
|        Spike Lee|   16|
|       Tim Burton|   16|
|Steven Soderbergh|   16|
|     Renny Harlin|   15|
|     Oliver Stone|   14|
+-----------------+-----+
only showing top 10 rows



# Truy vấn 2: Số lượng phim hành động trong 1 năm


In [5]:
from pyspark.sql.functions import col
action_movies_per_year = movies_df.filter(movies_df.Genres.contains("Action")) \
                                   .groupBy("year") \
                                   .count()

# Top 10
action_movies_per_year.orderBy("count", ascending=False).show(10)

+------+-----+
|  year|count|
+------+-----+
|2014.0|   71|
|2011.0|   54|
|2010.0|   53|
|2015.0|   53|
|2012.0|   50|
|2013.0|   47|
|2008.0|   45|
|2005.0|   45|
|2009.0|   44|
|2001.0|   44|
+------+-----+
only showing top 10 rows



# Truy vấn 3: Trung bình rating của các phim của một đạo diễn


In [6]:
director_avg_rating = movies_df.groupBy("DirectorName") \
                                .avg("Rating") \
                                .withColumnRenamed("avg(Rating)", "avg_rating")

# Top 10
director_avg_rating.orderBy("avg_rating", ascending=False).show(10)


+-----------------+-----------------+
|     DirectorName|       avg_rating|
+-----------------+-----------------+
| Sadyk Sher-Niyaz|8.699999809265137|
|        Cary Bell|8.699999809265137|
|     Mike Mayhall|8.600000381469727|
|  Charles Chaplin|8.600000381469727|
|       Raja Menon|              8.5|
|  Damien Chazelle|              8.5|
|     Majid Majidi|              8.5|
|       Ron Fricke|              8.5|
|     Sergio Leone|8.474999904632568|
|Christopher Nolan|8.425000071525574|
+-----------------+-----------------+
only showing top 10 rows

